In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os.path as op
import glob
import matplotlib.pyplot as plt
import ast
# import moss
import csv
import random
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from matplotlib.ticker import PercentFormatter
from statsmodels.stats.anova import AnovaRM
from sklearn.preprocessing import StandardScaler
from matplotlib.cm import get_cmap
from plot_utils import combined_rolling_average_and_model_fit, combined_sliding_window_and_model_fit_split_col
from common_functions import remove_unit_variance
scaler = StandardScaler()

df = pd.read_csv('pilot_total_clean_quest_updated.csv')
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
# df_cleaned = pd.read_csv('pilot_total_errors.csv')

/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T/ipykernel_65361/1060765235.py:29: DtypeWarning: Columns (254,255,256,257,258,259) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('pilot_total_clean_quest_updated.csv')


In [1]:
home_dir = op.abspath('./')

parent_dir = op.dirname(home_dir)

file_path = op.join(parent_dir, 'data', 'demographics.csv')

questionnaires = pd.read_csv(file_path)

NameError: name 'op' is not defined

In [3]:
questionnaires['StartDate'] = pd.to_datetime(
    questionnaires['StartDate'], 
    format='%Y-%m-%d %H:%M:%S', 
    errors='coerce'
)

# Remove invalid dates and filter
questionnaires = questionnaires.dropna(subset=['StartDate'])
pilot5 = questionnaires[
    (questionnaires['StartDate'] >= '2024-10-08') & 
    (questionnaires['StartDate'] <= '2024-10-30')
]

pilot6 = questionnaires[
    (questionnaires['StartDate'] >= '2024-11-22') & 
    (questionnaires['StartDate'] <= '2025-01-30')
]

In [4]:
### pilot5 '2024-10-08', '2024-10-30'
### pilot6 '2024-11-22', '2025-01-30'

In [5]:
pilot6


,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,Q1,BOR,Q22,Q34,Q5,Q25,Q26,Q27,Q28,id
1500,2024-11-22 23:54:48,2024-11-22 23:56:40,0,100,112,1,2024-11-22 23:56:41,R_3AYD67rz2zrQbgo,anonymous,EN,1,NaN,1,2,1,female,10/18/2003,"2,6",1,141181
1501,2024-11-23 00:24:25,2024-11-23 00:26:31,0,100,126,1,2024-11-23 00:26:32,R_1QRR55txOCGFEfI,anonymous,EN,1,NaN,1,NaN,1,female,12/26/2003,2,2,166588
1502,2024-11-23 17:23:09,2024-11-23 17:25:31,0,100,141,1,2024-11-23 17:25:32,R_5R4UcEFdKO8nYFH,anonymous,EN,1,NaN,1,2,1,female,12/27/2005,3,2,168691
1503,2024-11-24 00:19:03,2024-11-24 00:20:56,0,100,113,1,2024-11-24 00:20:57,R_7xBxRelclqvl2hz,anonymous,EN,1,NaN,1,2,1,female,09/06/2004,2,2,145849
1504,2024-11-24 12:49:02,2024-11-24 12:50:16,0,100,74,1,2024-11-24 12:50:17,R_1B9UyonAgDcJNtU,anonymous,EN,1,NaN,1,NaN,1,female,11/08/2004,5,1,167875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902,2025-01-28 19:59:01,2025-01-28 23:29:01,0,0,12600,0,2025-02-04 23:29:08,R_6PemmLv907dxuNr,anonymous,EN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,171031
1903,2025-01-29 19:21:05,2025-01-29 19:23:14,0,21,129,0,2025-02-05 19:23:14,R_70rp5L9kWTHnUNX,anonymous,EN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,161086
1904,2025-01-29 20:13:55,2025-01-29 20:14:47,0,57,51,0,2025-02-05 20:14:53,R_54l2LukOFB8na7r,anonymous,EN,1,NaN,1,NaN,2,NaN,NaN,NaN,NaN,145420
1905,2025-01-29 21:40:59,2025-01-29 21:47:18,0,29,379,0,2025-02-05 21:47:22,R_5h5YHmsVQMIS4en,anonymous,EN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,127165


In [6]:
home_dir = op.abspath('./')

data_files = glob.glob(op.join(home_dir,
                        'data',
                        '*.csv'))
sns.set_context('talk')

def df_creation(data_files, start_date, end_date):
    processed_dfs = []
    date_column = 'date'

    for file_path in data_files:
        try:
            temp_df = pd.read_csv(file_path)
            temp_df['filename'] = file_path
            processed_dfs.append(temp_df)
        except Exception as e:
            
            continue

    if processed_dfs:
        df = pd.concat(processed_dfs, ignore_index=True)
        df[date_column] = pd.to_datetime(df[date_column], format='%Y-%m-%d_%Hh%M.%S.%f', errors='coerce')
        df.dropna(subset=[date_column], inplace=True)
        df = df[(df[date_column] >= pd.to_datetime(start_date)) & (df[date_column] <= pd.to_datetime(end_date))]
        
        # # Uncomment and adapt the following lines as needed:
        # df = df.loc[df['V2_diff'].notnull()].reset_index(drop=True)
        # df['reliability'] = df['reliability'].astype(float)
        # df['Retrocue Reliability'] = np.where(df['reliability'] > 0.75, 'high', 'low')
        # non_numeric_values = df['resp_correct'][~df['resp_correct'].apply(np.isreal)]
        # numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
        # result = df[numeric_columns].groupby('participant').mean()
        
        

    else:
        # Return an empty DataFrame if no files were processed successfully
        df = pd.DataFrame()
    
    return df, numeric_columns

In [7]:
df_pilot5, numeric_columns = df_creation(data_files,'2024-10-08', '2024-10-30')


In [8]:
df_pilot6, numeric_columns = df_creation(data_files,'2024-11-22', '2025-01-30')


In [9]:
len(set(df_pilot5['participant']))

167

In [10]:
len(set(df_pilot6['participant']))

178

In [11]:
df_filtered_pilot5 = df[df['pilot_number'] == 1]
len(set(df_filtered_pilot5['participant']))

122

In [12]:
df_filtered_pilot6 = df[df['pilot_number'] == 2]
len(set(df_filtered_pilot6['participant']))

124

In [13]:
unique_participants_pilot5 = df_filtered_pilot5['participant'].unique()
len(unique_participants_pilot5)

122

In [14]:
unique_participants_pilot6 = df_filtered_pilot6['participant'].unique()
len(unique_participants_pilot6)

124

In [15]:
matched_rows = questionnaires[questionnaires['id'].isin(unique_participants_pilot5)]
matched_rows

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,Q1,BOR,Q22,Q34,Q5,Q25,Q26,Q27,Q28,id


In [16]:
matched_rows = questionnaires[questionnaires['id'].isin(unique_participants_pilot6)]
matched_rows

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,Q1,BOR,Q22,Q34,Q5,Q25,Q26,Q27,Q28,id


In [17]:
len(unique_participants_pilot6)

124

In [18]:
# Match participants and keep only the first entry for each
matched_rows_pilot5 = questionnaires[questionnaires['id'].astype(int).isin(unique_participants_pilot5)]
filtered_demo_pilot5 = matched_rows_pilot5.drop_duplicates(subset=['id'], keep='first')

matched_rows_pilot6 = questionnaires[questionnaires['id'].astype(int).isin(unique_participants_pilot6)]
filtered_demo_pilot6 = matched_rows_pilot6.drop_duplicates(subset=['id'], keep='first')


In [19]:
filtered_demo_pilot5['Q25'].unique()

array(['female', nan, 'male', 'decline', 'female ', 'Male', 'Female',
       'non-binary', 'Female ', 'f', 'deline'], dtype=object)

In [23]:
q25_counts_pilot6 = filtered_demo_pilot6['Q25'].value_counts()
q25_counts_pilot6

Q25
female        78
male          22
decline        4
Female         4
Male           4
female         3
nonbinary      1
femaile        1
non binary     1
male           1
Name: count, dtype: int64

In [24]:
q25_counts_pilot6 = filtered_demo_pilot6['Q25'].value_counts()
q25_counts_pilot6.sum()

119

In [25]:
# female - 97
# male - 28
# decline - 9
# non binary - 2

In [26]:
# female -108
# male - 17
# nonbinary - 3
# declined - 6

In [27]:
filtered_demo_pilot5['Q26'].unique()

array(['6/19/2003', 'decline', nan, '08/25/2003', '01/14/2005',
       '12/11/2002', 'January 17, 2005', '11/01/2004', '05/07/2004',
       '02/16/2005', '06/19/2005', '08/21/02', '04/17/2004', '03/03/2004',
       '03/18/2004', '09/06/2004', '10/26/2005', '11/07/2003',
       '08/16/2005', '04/11/2003', '12/12/2005', '10/18/2003',
       '06/13/2004', '12/15/2003', '11/06/2004', 'May 28th, 20004',
       '01/09/2004', '02/12/2005', '18 JAN 2024', '4 May 2005',
       '03/16/2003', '06/03/2000', '9/1/2004', '10/24/2003',
       'May 5th 2004', '04/28/2004', '8/28/2006', 'March 22, 2004',
       '01/18/2003', '01/30/2004', '09/15/2003', '10/29/2005', '09/06/04',
       '5/21/2005', '12/26/2003', '10/26/2004', 'Decline', '04/08/2005',
       '07/25/2006', '01/25/2003', '12/17/2002', '07/15/2003',
       '11/12/1991', '1//12/05', 'Feb/4/2002', '8/3/2005', '8/24/2005',
       '12/13/2004', '01-28-2003', '10/12/2004', '12/24/2003',
       '12/23/2002', '01/27/2004', '09/18/1999', '04262005'

In [99]:
import pandas as pd
from datetime import datetime

def process_birth_dates_with_start_dates(df, q26_column='Q26'):
    """
    Process birth dates and convert to ages using StartDate column in the same dataframe
    
    Parameters:
    df: DataFrame containing both birth dates (Q26) and StartDate columns
    q26_column: Name of the column containing birth dates (default 'Q26')
    
    Returns:
    Dictionary with results and processed DataFrame
    """
    
    df_copy = df.copy()
    
    # Clean invalid responses
    invalid_responses = ['decline', 'Decline', 'nan', '']
    df_copy['Q26_clean'] = df_copy[q26_column].replace(invalid_responses, pd.NA)
    
    # Fix common typos and format issues
    df_copy['Q26_clean'] = df_copy['Q26_clean'].str.replace('20004', '2004')
    df_copy['Q26_clean'] = df_copy['Q26_clean'].str.replace('1//12/05', '1/12/05')
    df_copy['Q26_clean'] = df_copy['Q26_clean'].str.replace('08212004', '08/21/2004')
    
    def parse_birth_dates(date_series):
        """Parse birth dates with multiple format attempts"""
        formats = [
            '%m/%d/%Y',      # 6/19/2003
            '%m/%d/%y',      # 08/21/02, 05/18/06
            '%Y/%m/%d',      # 2005/01/15
            '%m-%d-%Y',      # 02-11-2004, 12-24-2003
            '%m-%d-%y',      # 01-28-2003
            '%m %d %Y',      # 06 29 2005
            '%B %d, %Y',     # January 17, 2005, November 9, 2002
            '%b %d, %Y',     # May 5th 2004
            '%B %dth, %Y',   # July 19th, 2003
            '%B %drd, %Y',   # October 23rd, 2005
            '%d %B %Y',      # 18 JAN 2024, 10 October 2002
            '%B %d %Y',      # June 19 2004
            '%Y'             # 2005, 2003
        ]
        
        parsed_dates = pd.Series(index=date_series.index, dtype='datetime64[ns]')
        
        for idx, date_str in date_series.dropna().items():
            if pd.isna(parsed_dates[idx]):
                # Clean common suffixes and formatting
                clean_date = str(date_str).strip()
                clean_date = clean_date.replace('th,', ',').replace('rd,', ',').replace('nd,', ',').replace('st,', ',')
                clean_date = clean_date.replace('th ', ' ').replace('rd ', ' ').replace('nd ', ' ').replace('st ', ' ')
                
                # Try each format
                for fmt in formats:
                    try:
                        parsed_date = pd.to_datetime(clean_date, format=fmt)
                        parsed_dates[idx] = parsed_date
                        break
                    except:
                        continue
                
                # If still not parsed, try pandas' flexible parser as last resort
                if pd.isna(parsed_dates[idx]):
                    try:
                        parsed_dates[idx] = pd.to_datetime(clean_date, errors='coerce')
                    except:
                        continue
        
        return parsed_dates
    
    # Parse birth dates
    df_copy['birth_date'] = parse_birth_dates(df_copy['Q26_clean'])
    
    # Ensure StartDate is datetime (it's already in the dataframe)
    df_copy['StartDate'] = pd.to_datetime(df_copy['StartDate'], errors='coerce')
    
    # Calculate age for each participant using their individual StartDate
    df_copy['age'] = (df_copy['StartDate'] - df_copy['birth_date']).dt.days / 365.25
    
    # Generate results
    total_responses = len(df_copy)
    valid_birth_dates = df_copy['birth_date'].notna().sum()
    valid_start_dates = df_copy['StartDate'].notna().sum()
    valid_ages = df_copy['age'].dropna()
    
    failed_to_parse_birth = df_copy['Q26_clean'].notna().sum() - valid_birth_dates
    declined_or_missing_birth = df_copy['Q26_clean'].isna().sum()
    missing_start_dates = total_responses - valid_start_dates
    
    results = {
        'total_responses': total_responses,
        'valid_birth_dates': valid_birth_dates,
        'valid_start_dates': valid_start_dates,
        'failed_to_parse_birth': failed_to_parse_birth,
        'declined_or_missing_birth': declined_or_missing_birth,
        'missing_start_dates': missing_start_dates,
        'processed_df': df_copy
    }
    
    if len(valid_ages) > 0:
        # Identify and exclude the 1-year-old outlier
        one_year_old = df_copy[df_copy['age'] < 2]  # Find anyone under 2 years old
        if len(one_year_old) > 0:
            print(f"Found {len(one_year_old)} participant(s) under 2 years old - excluding from analysis:")
            print(one_year_old[['id', 'Q26', 'Q26_clean', 'birth_date', 'StartDate', 'age']])
            
            # Exclude the 1-year-old from age calculations
            df_copy_clean = df_copy[df_copy['age'] >= 2].copy()
            valid_ages_clean = df_copy_clean['age'].dropna()
            
            # Use cleaned ages for statistics
            if len(valid_ages_clean) > 0:
                results.update({
                    'age_mean': valid_ages_clean.mean(),
                    'age_sd': valid_ages_clean.std(),
                    'age_min': valid_ages_clean.min(),
                    'age_max': valid_ages_clean.max(),
                    'n_valid_ages': len(valid_ages_clean),
                    'excluded_1_year_old': True
                })
                results['processed_df'] = df_copy_clean
            else:
                # Fallback to original if no valid ages after exclusion
                results.update({
                    'age_mean': valid_ages.mean(),
                    'age_sd': valid_ages.std(),
                    'age_min': valid_ages.min(),
                    'age_max': valid_ages.max(),
                    'n_valid_ages': len(valid_ages)
                })
        else:
            # No outliers found, use all valid ages
            results.update({
                'age_mean': valid_ages.mean(),
                'age_sd': valid_ages.std(),
                'age_min': valid_ages.min(),
                'age_max': valid_ages.max(),
                'n_valid_ages': len(valid_ages)
            })
        
        # Show problematic entries
        problematic_birth = df_copy[df_copy['birth_date'].isna() & df_copy['Q26_clean'].notna()]
        if len(problematic_birth) > 0:
            results['problematic_birth_entries'] = problematic_birth['Q26_clean'].unique()
    
    return results

# Process both datasets
print("="*60)
print("PILOT 5 RESULTS")
print("="*60)

pilot5_results = process_birth_dates_with_start_dates(filtered_demo_pilot5)

print(f"Total responses: {pilot5_results['total_responses']}")
print(f"Valid birth dates parsed: {pilot5_results['valid_birth_dates']}")
print(f"Valid start dates: {pilot5_results['valid_start_dates']}")
print(f"Failed to parse birth dates: {pilot5_results['failed_to_parse_birth']}")
print(f"Declined or missing birth dates: {pilot5_results['declined_or_missing_birth']}")
print(f"Missing start dates: {pilot5_results['missing_start_dates']}")

if 'age_mean' in pilot5_results:
    excluded_note = " (excluding 1-year-old outlier)" if pilot5_results.get('excluded_1_year_old', False) else ""
    print(f"\nAge Statistics{excluded_note} (n = {pilot5_results['n_valid_ages']}):")
    print(f"Mean age: {pilot5_results['age_mean']:.1f} years")
    print(f"SD: {pilot5_results['age_sd']:.1f}")
    print(f"Range: {pilot5_results['age_min']:.0f}–{pilot5_results['age_max']:.0f}")
    
    # Demographic format
    print(f"\nDemographic format:")
    print(f"(M age = {pilot5_results['age_mean']:.1f} years, SD = {pilot5_results['age_sd']:.1f}, range = {pilot5_results['age_min']:.0f}–{pilot5_results['age_max']:.0f})")

if 'problematic_birth_entries' in pilot5_results:
    print(f"\nProblematic birth date entries:")
    print(pilot5_results['problematic_birth_entries'])

print("\n" + "="*60)
print("PILOT 6 RESULTS")
print("="*60)

pilot6_results = process_birth_dates_with_start_dates(filtered_demo_pilot6)

print(f"Total responses: {pilot6_results['total_responses']}")
print(f"Valid birth dates parsed: {pilot6_results['valid_birth_dates']}")
print(f"Valid start dates: {pilot6_results['valid_start_dates']}")
print(f"Failed to parse birth dates: {pilot6_results['failed_to_parse_birth']}")
print(f"Declined or missing birth dates: {pilot6_results['declined_or_missing_birth']}")
print(f"Missing start dates: {pilot6_results['missing_start_dates']}")

if 'age_mean' in pilot6_results:
    excluded_note = " (excluding 1-year-old outlier)" if pilot6_results.get('excluded_1_year_old', False) else ""
    print(f"\nAge Statistics{excluded_note} (n = {pilot6_results['n_valid_ages']}):")
    print(f"Mean age: {pilot6_results['age_mean']:.1f} years")
    print(f"SD: {pilot6_results['age_sd']:.1f}")
    print(f"Range: {pilot6_results['age_min']:.0f}–{pilot6_results['age_max']:.0f}")
    
    # Demographic format
    print(f"\nDemographic format:")
    print(f"(M age = {pilot6_results['age_mean']:.1f} years, SD = {pilot6_results['age_sd']:.1f}, range = {pilot6_results['age_min']:.0f}–{pilot6_results['age_max']:.0f})")

if 'problematic_birth_entries' in pilot6_results:
    print(f"\nProblematic birth date entries:")
    print(pilot6_results['problematic_birth_entries'])

# Store processed dataframes for further use
filtered_demo_pilot5_with_ages = pilot5_results['processed_df']
filtered_demo_pilot6_with_ages = pilot6_results['processed_df']

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Pilot 5: {pilot5_results.get('n_valid_ages', 0)} valid ages from {pilot5_results['total_responses']} responses")
print(f"Pilot 6: {pilot6_results.get('n_valid_ages', 0)} valid ages from {pilot6_results['total_responses']} responses")

PILOT 5 RESULTS
Found 1 participant(s) under 2 years old - excluding from analysis:
          id          Q26    Q26_clean birth_date           StartDate      age
1164  138742  18 JAN 2024  18 JAN 2024 2024-01-18 2024-10-14 15:42:02  0.73922
Total responses: 134
Valid birth dates parsed: 111
Valid start dates: 134
Failed to parse birth dates: 1
Declined or missing birth dates: 22
Missing start dates: 0

Age Statistics (excluding 1-year-old outlier) (n = 110):
Mean age: 20.7 years
SD: 2.0
Range: 18–33

Demographic format:
(M age = 20.7 years, SD = 2.0, range = 18–33)

Problematic birth date entries:
['04262005']

PILOT 6 RESULTS
Total responses: 136
Valid birth dates parsed: 110
Valid start dates: 136
Failed to parse birth dates: 0
Declined or missing birth dates: 26
Missing start dates: 0

Age Statistics (n = 110):
Mean age: 20.7 years
SD: 2.2
Range: 18–35

Demographic format:
(M age = 20.7 years, SD = 2.2, range = 18–35)

SUMMARY
Pilot 5: 110 valid ages from 134 responses
Pilot 6: 110